# 회귀

## 학생성적 예측 데이터

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv")


display(x_train.head())
display(y_train.head())

,StudentID,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,1714,GP,F,18,U,GT3,T,4,3,other,...,no,4,3,3,1,1,3,0,14,13
1,1254,GP,F,17,U,GT3,T,4,3,health,...,yes,4,4,3,1,3,4,0,13,15
2,1639,GP,F,16,R,GT3,T,4,4,health,...,no,2,4,4,2,3,4,6,10,11
3,1118,GP,M,16,U,GT3,T,4,4,services,...,no,5,3,3,1,3,5,0,15,13
4,1499,GP,M,19,U,GT3,T,3,2,services,...,yes,4,5,4,1,1,4,0,5,0


,StudentID,G3
0,1714,14
1,1254,15
2,1639,11
3,1118,13
4,1499,0


In [5]:
# print(x_train.isna().sum().sum())  ## 결측값 없음
print(x_train.info())
print(x_train.nunique())
# object columns 더미화 필요

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   StudentID   678 non-null    int64 
 1   school      678 non-null    object
 2   sex         678 non-null    object
 3   age         678 non-null    int64 
 4   address     678 non-null    object
 5   famsize     678 non-null    object
 6   Pstatus     678 non-null    object
 7   Medu        678 non-null    int64 
 8   Fedu        678 non-null    int64 
 9   Mjob        678 non-null    object
 10  Fjob        678 non-null    object
 11  reason      678 non-null    object
 12  guardian    678 non-null    object
 13  traveltime  678 non-null    int64 
 14  studytime   678 non-null    int64 
 15  failures    678 non-null    int64 
 16  schoolsup   678 non-null    object
 17  famsup      678 non-null    object
 18  paid        678 non-null    object
 19  activities  678 non-null    object
 20  nursery   

In [8]:
## drop
x_train_drop = x_train.drop(columns='StudentID')
x_test_drop = x_test.drop(columns='StudentID')
## dummies
x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]
y = y_train.G3

## split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np

xt, xv, yt, yv = train_test_split(x_train_dummies, y, test_size=0.3, random_state=0)
rf = RandomForestRegressor(random_state=0)
rf.fit(xt, yt)

## predict
yv_pred = rf.predict(xv)

## 모델 평가
# mse : mean_squared_error
# mae : mean_absolute_error 
# mape : mean_absolute_percentage_error
# rmse : root_mean_squerd_error -> 패키지 없음 np.sqrt(mean_squared_error)
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
print('== val ==')
print('mean_squared_error :', mean_squared_error(yv, yv_pred))
print('mean_absolute_error :', mean_absolute_error(yv, yv_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yv, yv_pred))
print('rmse :', np.sqrt(mean_squared_error(yv, yv_pred)))
print('r2_score :', r2_score(yv, yv_pred))

pred = rf.predict(x_test_dummies)
pd.DataFrame({'StudentID':x_test.StudentID, 'G3':pred}).to_csv('result_regression.csv', index=False)

== val ==
mean_squared_error : 1.1072933743191722
mean_absolute_error : 0.7524191176470587
mean_absolute_percentage_error : 437776375542926.2
rmse : 1.0522800835895223
r2_score : 0.9286134000516384


## 중고차 가격 예측 데이터

In [9]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv")


display(x_train.head())
display(y_train.head())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


In [17]:
print(x_train[['brand','model','transmission','fuelType']].nunique(), '\n',x_test[['brand','model','transmission','fuelType']].nunique())
## drop model

brand            9
model           90
transmission     4
fuelType         5
dtype: int64 
 brand            9
model           89
transmission     4
fuelType         5
dtype: int64


In [18]:
pd.concat([x_train_drop, x_test_drop])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,GP,F,18,U,GT3,T,4,3,other,other,...,no,4,3,3,1,1,3,0,14,13
1,GP,F,17,U,GT3,T,4,3,health,other,...,yes,4,4,3,1,3,4,0,13,15
2,GP,F,16,R,GT3,T,4,4,health,teacher,...,no,2,4,4,2,3,4,6,10,11
3,GP,M,16,U,GT3,T,4,4,services,services,...,no,5,3,3,1,3,5,0,15,13
4,GP,M,19,U,GT3,T,3,2,services,at_home,...,yes,4,5,4,1,1,4,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,GP,F,15,R,LE3,T,3,1,other,other,...,no,4,4,2,2,3,3,6,15,15
362,GP,M,17,U,GT3,T,3,3,services,services,...,yes,4,3,4,2,3,4,12,12,12
363,GP,F,16,U,GT3,T,2,3,services,teacher,...,no,2,3,1,1,1,3,2,16,16
364,MS,F,16,U,GT3,T,3,1,other,other,...,no,3,1,3,1,3,1,0,8,6


In [14]:
print(x_test.info(), x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         2672 non-null   int64  
 1   brand         2672 non-null   object 
 2   model         2672 non-null   object 
 3   year          2672 non-null   int64  
 4   transmission  2672 non-null   object 
 5   mileage       2672 non-null   int64  
 6   fuelType      2672 non-null   object 
 7   tax           2672 non-null   float64
 8   mpg           2672 non-null   float64
 9   engineSize    2672 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 208.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4960 entries, 0 to 4959
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   carID         4960 non-null   int64  
 1   brand         4960 non-null   object 
 2   model         4960 non-null  

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

x_train_drop = x_train.drop(columns = 'carID')
x_test_drop = x_test.drop(columns = 'carID')
y = y_train['price']

# train에만 있고 test에는 없는 model들이 있기에 합쳐서 원핫 인코딩을 진행
combined = pd.concat([x_train_drop, x_test_drop])
combined_encoded = pd.get_dummies(combined)
train_encoded = combined_encoded[:len(x_train_drop)]
test_encoded = combined_encoded[len(x_train_drop):]

xt , xv , yt, yv = train_test_split(train_encoded , y, test_size=0.3, random_state=0)
rf = RandomForestRegressor(random_state =42)
rf.fit(xt, yt)

yv_pred = rf.predict(xv)

print('== val ==')
print('mean_squared_error :', mean_squared_error(yv, yv_pred))
print('mean_absolute_error :', mean_absolute_error(yv, yv_pred))
print('mean_absolute_percentage_error :', mean_absolute_percentage_error(yv, yv_pred))
print('rmse :', np.sqrt(mean_squared_error(yv, yv_pred)))
print('r2_score :', r2_score(yv, yv_pred))

pred = rf.predict(test_encoded)
pd.DataFrame({'carID':x_test.carID, 'price':pred}).to_csv('result_regression.csv', index=False)

== val ==
mean_squared_error : 12861197.338756274
mean_absolute_error : 2077.9584928207432
mean_absolute_percentage_error : 0.10592654437084767
rmse : 3586.251153887061
r2_score : 0.9538282515270621


## 의료 비용 예측 데이터